# - Clustering -

# Data Preparation

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
# Memasukkan Data
df = pd.read_csv('../input/customer-personality-analysis/marketing_campaign.csv', sep='\t')

In [3]:
# Menampilkan 5 Data teratas
df.head()

# Data Cleaning

In [4]:
# Melihat Info dari setiap kolom
df.info()

In [5]:
# Mengecek apakah terdapat nilai NULL atau tidak
df.isnull().sum()

In [6]:
# Total Baris dan Kolom sebelum di Hapus nilai NULL
print('Baris dan Kolom sebelum di hapus nilai NULL:', df.shape)

# Menghapus nilai NULL pada Baris
df.dropna(axis=0, inplace=True)

# Total Baris dan Kolom sesudah di Hapus nilai NULL
print('Baris dan Kolom sesudah di hapus nilai NULL:', df.shape)

# Data Transformation and Visualization

In [7]:
from datetime import date

In [8]:
# Transformasi Year_Birth menjadi Age
df['Age'] = date.today().year - df['Year_Birth']

# Memperbaiki format tipe Dt_Customer dari objek ke waktu
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])
# Mengecek apakah sudah menjadi tipe data waktu
print('Tipe kolom Dt_Customer:', df['Dt_Customer'].dtypes)

# Transformasi Dt_Customer menjadi Year_Enroll
df['Year_Enroll'] = pd.DatetimeIndex(df['Dt_Customer']).year
df['Enroll_Len'] = date.today().year - pd.DatetimeIndex(df['Dt_Customer']).year

# Mengubah tipe data Income float menjadi integer
df['Income'] = df['Income'].astype('int')

In [9]:
# Transformasi Kidhome dan Teenhome menjadi Children
df['Children'] = df['Kidhome']+df['Teenhome']

# Transformasi Mnt[Products] menjadi Spending (Pengeluaran)
df['Spending'] = np.sum(df.iloc[:, 9:14].values, axis=1)

# Mengganti Menyederhanakan kategori pada Pendidikan dan Status Pernikahan
df['Education'] = df['Education'].replace(dict.fromkeys(['Basic','2n Cycle'], 'Undergraduate')).replace(dict.fromkeys(['Graduation','Master','PhD'], 'Postgraduate'))
df['Marital_Status'] = df['Marital_Status'].replace(dict.fromkeys(['Divorced','Single','Absurd','Widow','YOLO'], 'Alone')).replace(dict.fromkeys(['Married','Together'], 'Couples'))

In [10]:
# Mengganti data lama dan Memilih data yang akan digunakan nanti
df = df[['ID','Age','Education','Marital_Status','Income','Children','Year_Enroll','Enroll_Len','Spending','MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds','Complain']]
# Menampilkan 5 Data teratas
df.head()

In [11]:
# Deskripsi Data setelah Pembersihan
df.select_dtypes('int').describe()

In [12]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Image
import os
os.makedirs('./chart')

In [13]:
conda install -c plotly python-kaleido

In [14]:
dirs_img = './chart/'

In [15]:
# Chart Jumlah Customer berdasarkan Komplain atau Tidak
fig_complain = px.histogram(df, x='Complain', y='ID', 
                            color='Complain', histfunc='count', text_auto='auto',
                            title='Chart Jumlah Customer berdasarkan Komplain atau Tidak')
fig_complain.write_image(dirs_img+'cpa_customer_by_complain.png')
Image(dirs_img+'cpa_customer_by_complain.png')

In [16]:
# Proporsi Customer berdasarkan Pendidikan dan Status Pernikahan
df_edu = df.groupby(by=['Education']).size().reset_index(name='counts')
df_marstat = df.groupby(by=['Marital_Status']).size().reset_index(name='counts')

fig_proportion = make_subplots(rows=1, cols=2, 
                               specs=[[{"type": "pie"}, {"type": "pie"}]], 
                               subplot_titles=('Pendidikan',  'Status Pernikahan'))

fig_proportion.add_trace(go.Pie(values=df_edu['counts'],
                                labels=df_edu['Education'],
                                name='Pendidikan',
                                legendgroup='1',
                                legendgrouptitle_text='Pendidikan'),
                                row=1, col=1)

fig_proportion.add_trace(go.Pie(values=df_marstat['counts'],
                                labels=df_marstat['Marital_Status'],
                                name='Status Pernikahan', 
                                legendgroup='2',
                                legendgrouptitle_text='Status Pernikahan'), 
                                row=1, col=2)

fig_proportion.update_layout(title_text='Proporsi Customer berdasarkan Pendidikan dan Status Pernikahan',
                             legend_tracegroupgap=25)
fig_proportion.write_image(dirs_img+'cpa_proporsion_customer.png')
Image(dirs_img+'cpa_proporsion_customer.png')

In [17]:
# Chart Rata-rata Pendapatan Customer berdasarkan Tahun saat mendaftar
df_mean_income = df.groupby(['Year_Enroll'])['Income'].mean().reset_index(name='Avg_Income')
fig_income = go.Figure()
fig_income.add_trace(go.Scatter(x=df_mean_income['Year_Enroll'], 
                                y=df_mean_income['Avg_Income'], 
                                mode='lines+markers'))
fig_income.update_layout(xaxis=dict(tickvals=df_mean_income['Year_Enroll'].values,
                                    showline=True, ticks='outside', linecolor='rgb(204, 204, 204)', linewidth=2,),
                         yaxis=dict(showline=True, ticks='outside', linecolor='rgb(204, 204, 204)', linewidth=2,),
                         title='Rata-rata Pendapatan Customer berdasarkan Tahun saat mendaftar',
                         plot_bgcolor='white')
fig_income.write_image(dirs_img+'cpa_avg_income_customer.png')
Image(dirs_img+'cpa_avg_income_customer.png')

In [18]:
from sklearn.preprocessing import LabelEncoder, RobustScaler, normalize

In [19]:
# Mengkodekan kolom kategori yang bertipe Object menjadi Numerik
le = LabelEncoder()

df['Education'] = le.fit_transform(df['Education'])
df['Marital_Status'] = le.fit_transform(df['Marital_Status'])

# Fitur Skala pada Age, Income, Enroll_Len, dan Spending. kemudian di normalisasikan
rs = RobustScaler()
X_rs = rs.fit_transform(df[['Income','Spending','Age']])
X_norm = normalize(X_rs, norm='l2')

# Modelling

In [20]:
from sklearn.cluster import KMeans

Disini saya akan membuat klaster 3 dimensi dengan 4 Klaster:
1. **Tingkat 1** = Pendapatan dan pengeluaran rata-ratanya tinggi dengan jumlah lulusan pendidikan sarjana dan pascasarjana banyak
2. **Tingkat 2** = Pendapatan dan pengeluaran rata-ratanya rendah dengan jumlah lulusan pendidikan sarjana dan pascasarjana banyak
3. **Tingkat 3** = Pendapatan rata-ratanya sedang dan pengeluaran rata-ratanya rendah dengan jumlah lulusan pendidikan sarjana dan pascasarjana sedikit
4. **Tingkat 4** = Pendapatan dan pengeluaran rata-ratanya rendah dengan jumlah lulusan pendidikan sarjana dan pascasarjana sedikit

In [21]:
# Menjalankan pemodelan dengan jumlah klasternya 4
model = KMeans(n_clusters=4, random_state=5, max_iter=1000).fit(X_norm)
label = model.predict(X_norm)

# Memasukkan hasil label dari pemodelan dengan membuat kolom baru Cluster
df['Cluster'] = label
df['Cluster'] = df['Cluster'].replace({0:'Tingkat 1',1:'Tingkat 2',2:'Tingkat 3',3:'Tingkat 4'})

result_cluster = df[['Income','Spending','Education','Cluster']]
result_cluster.set_index('Cluster', inplace = True)

# Menampilkan Deskripsi data hasil cluster
result_cluster.groupby('Cluster').describe().transpose()

In [22]:
# Melihat terlebih dahulu proporsi dari masing-masing cluster
df_cluster = df.groupby(by=['Cluster']).size().reset_index(name='counts')
fig_cluster = px.pie(df_cluster, values='counts', names='Cluster', title='Proporsi dari Masing-masing Klaster')
fig_cluster.write_image(dirs_img+'cpa_proportion_of_cluster.png')
Image(dirs_img+'cpa_proportion_of_cluster.png')

In [23]:
# Pivot tabel
df.pivot_table(index='Cluster',aggfunc='mean')

In [24]:
# Memplotkan hasil dari cluster yang sudah didapatkan dari pemodelan berdasarkan Income, Spending, dan Education
plot_model = go.Figure()
for cluster in list(df.Cluster.unique()):
    plot_model.add_trace(go.Scatter3d(x=df[df.Cluster == cluster]['Education'],
                                      y=df[df.Cluster == cluster]['Income'],
                                      z=df[df.Cluster == cluster]['Spending'],                        
                                      mode='markers', marker_size=6, marker_line_width=1,
                                      name=str(cluster)))

plot_model.update_traces(hovertemplate='Pendidikan: %{x} <br>Pendapatan: %{y} <br>Pengeluaran: %{z}')
plot_model.update_layout(width=750, height=750, autosize=True, showlegend=True,
                             scene=dict(xaxis=dict(title='Pendidikan'),
                                        yaxis=dict(title='Pendapatan'),
                                        zaxis=dict(title='Pengeluaran')),
                             title='Klasterisasi Customer berdasarkan Pendidikan, Pendapatan, dan Pengeluaran')
plot_model.write_image(dirs_img+'cpa_plot_model.png')
Image(dirs_img+'cpa_plot_model.png')

# Conclusion
Berdasarkan dari hasil output diatas dapat disimpulkan bahwa:
1. Berdasarkan output ke-19 dan 20. Dari 2216 Customer yang telah dikelompokkan menjadi 4 Cluster, terlihat bahwa proporsi Cluster Tingkat 1 yang paling banyak Customernya dibandingkan Cluster lainnya.
2. Rata-rata Pendapatan dan Pengeluaran Customer yang didapat pada masing-masing Cluster yakni,
    * Tingkat 1 sebesar 70688.23 USD dan 1080.69 USD
    * Tingkat 2 sebesar 33883.63 USD dan 115.02 USD
    * Tingkat 3 sebesar 58552.95 USD dan 162.48 USD
    * Tingkat 4 sebesar 32295.1 USD dan 144.42 USD
3. Berdasarkan output ke-21. Pada Cluster pendapatan dan pengeluaran rata-ratanya tinggi dengan jumlah lulusan pendidikan sarjana dan pascasarjana banyak, lebih banyak uang yang mereka harus keluarkan untuk membeli produk seperti Wine, Fruit, Meat, Fish, Sweet, dan Gold.
4. Output ke-22 merupakan Grafik Penyebaran yang sudah sesuai di Klasterisasikan berdasarkan Pendidikan, Pendapatan, dan Pengeluaran.